In [4]:
num = 5
num

5

In [20]:
import sys

import pandas as pd

print(sys.executable)


/Users/ericwiniecke/.pyenv/versions/3.12.4/envs/cost_cup_env/bin/python


In [22]:
import os

import altair as alt
import altair_viewer

alt.renderers.enable("default")
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv()

# Database connection parameters
DATABASE_TYPE = os.getenv("DATABASE_TYPE")
DBAPI = os.getenv("DBAPI")
ENDPOINT = os.getenv("ENDPOINT")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
PORT = int(os.getenv("PORT", 5432))
DATABASE = os.getenv("DATABASE", "hockey_stats")


# Create the connection string
connection_string = (
    f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
)
engine = create_engine(connection_string)

# Define the seasons to query
seasons = ["20152016", "20162017", "20172018"]

# Initialize an empty DataFrame to collect all the data
df = pd.DataFrame()

# Loop through each season and query the corresponding table
for season in seasons:
    query = f"""
    SELECT "capHit", "CF_Percent"
    FROM public.aggregated_corsi_{season}
    """
    season_df = pd.read_sql(query, engine)
    season_df["season"] = season  # Add the season column manually in Python
    df = pd.concat([df, season_df], ignore_index=True)

# Convert `capHit` and `CF_Percent` to numeric if they aren't already
df["capHit"] = pd.to_numeric(df["capHit"], errors="coerce")
df["CF_Percent"] = pd.to_numeric(df["CF_Percent"], errors="coerce")

# Drop any null values that might exist after conversion
df = df.dropna(subset=["capHit", "CF_Percent"])

# Create scatter plots for each season using Altair
charts = []

for season in seasons:
    # Filter data for the specific season and ensure it remains a DataFrame
    df_season = df[df["season"] == season]

    # Ensure df_season is still a DataFrame
    if isinstance(df_season, pd.Series):
        df_season = df_season.to_frame()

    # Ensure the DataFrame isn't empty
    if df_season.empty:
        print(f"No data available for season {season}")
        continue

    # Create the scatter plot with Altair
    scatter_plot = (
        alt.Chart(df_season)
        .mark_circle(size=60)
        .encode(
            x=alt.X("CF_Percent", title="Corsi For Percentage (CF%)"),
            y=alt.Y("capHit", title="Cap Hit ($)"),
            tooltip=["CF_Percent", "capHit"],
            opacity=alt.value(0.6),
        )
        .properties(title=f"Season {season}", width=250, height=300)
    )

    charts.append(scatter_plot)

# Combine the charts into a horizontal layout
combined_chart = alt.concat(*charts, columns=3)
# Save the chart as an HTML file
combined_chart.save("combined_chart.html")

# Alternatively, display the chart directly in a browser
altair_viewer.show(combined_chart)
# Show the plot
# combined_chart.display()
altair_viewer.show(combined_chart, open_browser=True)

OperationalError: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  role "postgresql" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [12]:
print(df.head())

Empty DataFrame
Columns: []
Index: []


In [16]:
"""
August 11, 2024
Code for chart displaying exploratory
data exploration.
Eric Winiecke
"""

import os

import pandas as pd
from dotenv import load_dotenv
from matplotlib import pyplot as plt
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv()

# Database connection parameters
DATABASE_TYPE = os.getenv("DATABASE_TYPE")
DBAPI = os.getenv("DBAPI")
ENDPOINT = os.getenv("ENDPOINT")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
PORT = int(os.getenv("PORT", 5432))
DATABASE = os.getenv("DATABASE", "hockey_stats")

# Create the connection string
connection_string = (
    f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
)
engine = create_engine(connection_string)

# Define the seasons to query
seasons = ["20152016", "20162017", "20172018"]

# Initialize an empty DataFrame to collect all the data
df = pd.DataFrame()

# Loop through each season and query the corresponding table
for season in seasons:
    query = f"""
    SELECT "capHit", "CF_Percent"
    FROM public.aggregated_corsi_{season}
    """
    season_df = pd.read_sql(query, engine)
    season_df["season"] = season  # Add the season column manually in Python
    df = pd.concat([df, season_df], ignore_index=True)

# Convert `capHit` and `CF_Percent` to numeric if they aren't already
df["capHit"] = pd.to_numeric(df["capHit"], errors="coerce")
df["CF_Percent"] = pd.to_numeric(df["CF_Percent"], errors="coerce")

# Drop any null values that might exist after conversion
df = df.dropna(subset=["capHit", "CF_Percent"])

# Create subplots for each season
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6), sharey=True, sharex=True)

# Loop through each season and create a scatter plot in its own subplot
for ax, season in zip(axes, seasons):
    # Filter data for the specific season
    df_season = df[df["season"] == season]

    # Create the scatter plot with CF_Percent on x-axis and capHit on y-axis
    ax.scatter(
        df_season["CF_Percent"],
        df_season["capHit"],
        alpha=0.6,
        edgecolors="w",
        linewidth=0.5,
    )

    # Set title and labels
    ax.set_title(f"Season {season}")
    ax.set_xlabel("Corsi For Percentage (CF%)")
    ax.set_ylabel("Cap Hit ($)")
    ax.grid(True)

# Set the y-label only on the first subplot (since sharey=True)
axes[0].set_ylabel("Cap Hit ($)")

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

OperationalError: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  role "postgresql" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)